In [1]:
from flask import Flask, jsonify, request, abort, make_response
from flask.json import JSONEncoder

class Book:
    def __init__(self, isbn, title, author, price):
        self.isbn = isbn
        self.title = title
        self.author = author
        self.price = price
    def __str__(self):
        return self.title + ' by ' + self.author + ' @ ' + str(self.price)
    
class BookJSONEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Book):
            return {'isbn': obj.isbn, 'title': obj.title, 'author': obj.author, 'price': obj.price}
        else:
            return super(BookJSONEncoder, self).default(obj)
        
class Bookshop:
    def __init__(self, books):
        self.books = books
    def get(self, isbn):
        if int(isbn) > len(self.books):
            abort(404)
        return list(filter(lambda b: b.isbn == isbn, self.books))[0]
    def add_book(self, book):
        self.books.append(book)
    def delete_book(self, isbn):
        self.books = list(filter(lambda b: b.isbn != isbn, self.books))
        
bookshop = Bookshop([Book(1, 'XML', 'Gryff Smith', 10.99), Book(2, 'Java', 'Phoebe Cooke', 12.99),
                     Book(3, 'Scala', 'Adam Davies', 11.99), Book(4, 'Python', 'Jasmine Byrne', 15.99)])

def create_bookshop_service():
    app = Flask(__name__)
    app.json_encoder = BookJSONEncoder
    
    @app.route('/book/list', methods=['GET'])
    def get_books():
        return jsonify({'books': bookshop.books})

    @app.route('/book/<int:isbn>', methods=['GET'])
    def get_book(isbn):
        book = bookshop.get(isbn)
        return jsonify({'book': book})

    @app.route('/book', methods=['POST'])
    def create_book():
        print('create book')
        if not request.json or not 'isbn' in request.json:
            abort(400)
        book = Book(request.json['isbn'], request.json['title'], request.json.get('author', ""),
                    float(request.json['price']))
        bookshop.add_book(book)
        return jsonify({'book': book}), 201
    @app.route('/book', methods=['PUT'])
    def update_book():
        if not request.json or not 'isbn' in request.json:
            abort(400)
        isbn = request.json['isbn']
        book = bookshop.get(isbn)
        book.title = request.json['title']
        book.author = request.json['author']
        book.price = request.json['price']
        return jsonify({'book': book}), 201
    @app.route('/book/<int:isbn>', methods=['DELETE'])
    def delete_book(isbn):
        bookshop.delete_book(isbn)
        return jsonify({'result': True})
    @app.errorhandler(400)
    def not_found(error):
        return make_response(jsonify({'book': 'Not found'}), 400)
    return app

if __name__ == '__main__':
    app = create_bookshop_service()
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Jun/2021 13:38:06] "GET /stock/list HTTP/1.1" 404 -
127.0.0.1 - - [02/Jun/2021 13:38:16] "GET /book/list HTTP/1.1" 200 -
[2021-06-02 13:38:26,194] ERROR in app: Exception on /book/2 [GET]
Traceback (most recent call last):
  File "/home/jorge/.local/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/jorge/.local/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/jorge/.local/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/jorge/.local/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/jorge/.local/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/jorge/.loca

1
2
3
4


In [1]:
from flask import Flask, jsonify, request, abort, make_response
from flask.json import JSONEncoder

class Stock:
    def __init__(self, ticker, price):
        self.ticker = ticker
        self.price = price
    def __str__(self):
        return self.ticker + ':' + str(self.price)
    
class StockJSONEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Stock):
            return {obj.ticker: obj.price}
        else:
            return super(StockJSONEncoder, self).default(obj)

class Stock_Market:
    def __init__(self, stocks):
        self.stocks = stocks
    def get(self, ticker):
        return list(filter(lambda b: b.ticker == ticker, self.stocks))[0]
    def add_stock(self, stock):
        self.stocks.append(stock)
    def delete_stock(self, ticker):
        self.stocks = list(filter(lambda b: b.ticker != ticker, self.stocks))
        
stockmarket = Stock_Market([Stock('IBM', 12.55), Stock('APPL', 15.66), Stock('GOOG', 5.22)])

def create_stock_market_service():
    app = Flask(__name__)
    app.json_encoder = StockJSONEncoder
    
    @app.route('/stock/list', methods=['GET'])
    def get_stocks():
        return jsonify({'stocks': stockmarket.stocks})

    @app.route('/stock/<ticker>', methods=['GET'])
    def get_stock(ticker):
        stock = stockmarket.get(ticker)
        return jsonify({'stock': stock})

    @app.route('/stock', methods=['POST'])
    def create_stock():
        print('create stock')
        if not request.json or not 'ticker' in request.json:
            abort(400)
        stock = Stock(request.json['ticker'], float(request.json['price']))
        stockmarket.add_stock(stock)
        return jsonify({'stock': stock}), 201
    @app.route('/stock', methods=['PUT'])
    def update_stock():
        if not request.json or not 'ticker' in request.json:
            abort(400)
        ticker = request.json['ticker']
        stock = stockmarket.get(ticker)
        stock.price = request.json['price']
        return jsonify({'stock': stock}), 201
    @app.route('/stock/<ticker>', methods=['DELETE'])
    def delete_stock(ticker):
        stockmarket.delete_stock(ticker)
        return jsonify({'result': True})
    @app.errorhandler(400)
    def not_found(error):
        return make_response(jsonify({'stock': 'Not found'}), 400)
    return app

if __name__ == '__main__':
    app = create_stock_market_service()
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Jun/2021 13:58:36] "GET /stock/IBM HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2021 13:58:42] "PUT /stock HTTP/1.1" 400 -
127.0.0.1 - - [02/Jun/2021 13:58:45] "GET /stock/IBM HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2021 13:58:47] "GET /stock/IBM HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2021 13:58:59] "PUT /stock HTTP/1.1" 201 -
127.0.0.1 - - [02/Jun/2021 13:59:03] "GET /stock/IBM HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2021 13:59:54] "POST /stock HTTP/1.1" 201 -


create stock


127.0.0.1 - - [02/Jun/2021 14:00:02] "GET /stock/list HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2021 14:00:37] "DELETE /book/GOOG HTTP/1.1" 404 -
127.0.0.1 - - [02/Jun/2021 14:00:50] "DELETE /stock/GOOG HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2021 14:00:54] "GET /stock/list HTTP/1.1" 200 -
